# KSE EP-OpInf Tests

In [ ]:
using DataFrames
using FileIO
using JLD2
using LaTeXStrings
using LinearAlgebra
using Plots
using Plots.PlotMeasures
using ProgressMeter
using StatsBase

include("../src/model/KS.jl")
include("../src/LiftAndLearn.jl")
const LnL = LiftAndLearn

# Settings for the KS equation
KSE = KS(
    [0.0, 22.0], [0.0, 300.0], [1.0, 1.0],
    512, 0.001, 1, "ep"
)

# WARNING:DO YOU WANT TO SAVE DATA?
save_data = true

# Create file name to save data
datafile = "data/kse_data_L22.jld2"
testdatafile = "data/kse_test_data_L22.jld2"
opfile = "data/kse_operators_L22.jld2"
resultfile = "data/kse_results_L22.jld2"
testresultfile = "data/kse_test_results_L22.jld2"

# Downsampling rate
DS = 100

# Down-sampled dimension of the time data
Tdim_ds = size(1:DS:KSE.Tdim, 1)  # downsampled time dimension

# Number of random test inputs
num_test_ic = 50
;

In [ ]:
# Parameterized function for the initial condition
L = KSE.Omega[2] - KSE.Omega[1]  # length of the domain
u0 = (a,b) -> a * cos.((2*π*KSE.x)/L) .+ b * cos.((4*π*KSE.x)/L)  # initial condition
;  

In [ ]:
DATA = load(datafile);
TEST_DATA = load(testdatafile);
OPS = load(opsfile);

## Test 1 

In [ ]:
# Setup the results
TEST_RES = Dict{String, Any}()
TEST_RES["test1_AC"] = Dict(
    :int   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :LS    => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :ephec => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :fom   => Array{Array{Float64}}(undef, KSE.Pdim)
)
TEST_RES["test2_AC"] = Dict(
    :int   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :LS    => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :ephec => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :fom   => Array{Array{Float64}}(undef, KSE.Pdim)
)
TEST_RES["test1_LE"] = Dict(
    :int   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :LS    => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :ephec => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :epsic => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :epp   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :fom   => Array{Array{Float64}}(undef, KSE.Pdim)
)
TEST_RES["test2_LE"] = Dict(
    :int   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :LS    => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :ephec => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :epsic => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :epp   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :fom   => Array{Array{Float64}}(undef, KSE.Pdim)
)
TEST_RES["test1_DKY"] = Dict(
    :int   => Array{Float64}(undef, length(ro), KSE.Pdim),
    :LS    => Array{Float64}(undef, length(ro), KSE.Pdim),
    :ephec => Array{Float64}(undef, length(ro), KSE.Pdim),
    :epsic => Array{Float64}(undef, length(ro), KSE.Pdim),
    :epp   => Array{Float64}(undef, length(ro), KSE.Pdim),
    :fom   => Array{Float64}(undef, KSE.Pdim)
)
TEST_RES["test2_DKY"] = Dict(
    :int   => Array{Float64}(undef, length(ro), KSE.Pdim),
    :LS    => Array{Float64}(undef, length(ro), KSE.Pdim),
    :ephec => Array{Float64}(undef, length(ro), KSE.Pdim),
    :epsic => Array{Float64}(undef, length(ro), KSE.Pdim),
    :epp   => Array{Float64}(undef, length(ro), KSE.Pdim),
    :fom   => Array{Float64}(undef, KSE.Pdim)
)
TEST_RES["test1_LE_all"] = Dict(
    :int   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :LS    => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :ephec => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :epsic => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :epp   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :fom   => Array{Array{Float64}}(undef, KSE.Pdim)
)
TEST_RES["test2_LE_all"] = Dict(
    :int   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :LS    => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :ephec => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :epsic => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :epp   => Array{Array{Float64}}(undef, length(ro), KSE.Pdim),
    :fom   => Array{Array{Float64}}(undef, KSE.Pdim)
)
;

In [ ]:
lags = 0:KSE_data.DS:Int(floor(KSE.Tdim));

In [ ]:
# FOM
foo = zeros(length(lags))
bar = length(TEST_DATA["ic_a_rand_in"])
prog = Progress(bar)
for i in 1:bar
    foo .+= analyze_autocorr(KSE, TEST_DATA["Xtest_in"], i, lags)[1]
    next!(prog)
end
TEST_RES["test1_AC"][:fom][1] = foo / bar

In [ ]:
# Least-squares
foo = zeros(length(lags), length(DATA["ro"]))
bar = length(TEST_DATA["ic_a_rand_in"])
prog = Progress(bar)
for i in 1:bar
    a = TEST_DATA["ic_a_rand_in"][i]
    b = TEST_DATA["ic_b_rand_in"][i]

    tmp = analyze_autocorr(OPS["op_LS"], KSE, DATA["Vr"], u0(a,b), DATA["ro"], KSE.integrate_FD, lags)
    for j in eachindex(DATA["ro"])
        foo[:,j] .+= tmp[j,1]
    end
    next!(prog)
end

for j in eachindex(DATA["ro"])
    TEST_RES["test1_AC"][:LS][j,1] = foo[:,j] / bar
end

In [ ]:
# intrusive
foo = zeros(length(lags), length(DATA["ro"]))
bar = length(TEST_DATA["ic_a_rand_in"])
prog = Progress(bar)
for i in 1:bar
    a = TEST_DATA["ic_a_rand_in"][i]
    b = TEST_DATA["ic_b_rand_in"][i]

    tmp = analyze_autocorr(OPS["op_int"], KSE, DATA["Vr"], u0(a,b), DATA["ro"], KSE.integrate_FD, lags)
    for j in eachindex(DATA["ro"])
        foo[:,j] .+= tmp[j,1]
    end
    next!(prog)
end

for j in eachindex(DATA["ro"])
    TEST_RES["test1_AC"][:int][j,1] = foo[:,j] / bar
end

In [ ]:
# ep-opinf
foo = zeros(length(lags), length(DATA["ro"]))
bar = length(TEST_DATA["ic_a_rand_in"])
prog = Progress(bar)
for i in 1:bar
    a = TEST_DATA["ic_a_rand_in"][i]
    b = TEST_DATA["ic_b_rand_in"][i]

    tmp = analyze_autocorr(OPS["op_ephec"], KSE, DATA["Vr"], u0(a,b), DATA["ro"], KSE.integrate_FD, lags)
    for j in eachindex(DATA["ro"])
        foo[:,j] .+= tmp[j,1]
    end
    next!(prog)
end

for j in eachindex(DATA["ro"])
    TEST_RES["test1_AC"][:ephec][j,1] = foo[:,j] / bar
end

In [ ]:
lags_t = collect(lags) .* KSE.Δt
idx = length(lags_t) ÷ 2
lout = @layout [grid(2,2)]
p = plot(layout=lout, size=(1000, 500))
for (plot_id, ri) in enumerate([1, 2, 5, 7])
    plot!(p[plot_id], lags_t[1:idx], TEST_RES["test1_AC"][:LS][ri][1:idx], c=:black, label="opinf")
    plot!(p[plot_id], lags_t[1:idx], TEST_RES["test1_AC"][:int][ri][1:idx], c=:orange, label="intrusive")
    plot!(p[plot_id], lags_t[1:idx], TEST_RES["test1_AC"][:ephec][ri][1:idx], c=:blue, ls=:dash, lw=2, label="ephec-opinf")
    # plot!(p[plot_id], lags_t, TEST_RES["test1_AC"][:epsic][ri], c=:purple, ls=:dot, label="epsic-opinf")
    # plot!(p[plot_id], lags_t, TEST_RES["test1_AC"][:epp][ri], c=:red, lw=1, ls=:dash, label="epp-opinf")
    plot!(p[plot_id], lags_t[1:idx], TEST_RES["test1_AC"][:fom][1][1:idx], c=:green, lw=1, ls=:dash, label="fom")
    plot!(p[plot_id], fontfamily="Computer Modern", guidefontsize=12, tickfontsize=13)
    plot!(p[plot_id], title="r = $(DATA["ro"][ri])", titlefontsize=14, titlefontfamily="Computer Modern")
    # xlabel!(p[plot_id], "lag")
    # ylabel!(p[plot_id], "normalized autocorrelation")
    yticks!(p[plot_id], [-0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0])

    if plot_id == 4
        plot!(p[plot_id], legend=:topright, legend_column=2, legendfontsize=12)
    else
        plot!(p[plot_id], legend=false)
    end
end
plot!(p[1], left_margin=9mm)
plot!(p[3], bottom_margin=9mm, left_margin=9mm)
plot!(p[4], bottom_margin=9mm)
annotate!(p[4], -20, -0.48, "time lag", annotationfontsize=16)
annotate!(p[1], -30, -0.4, Plots.text("normalized autocorrelation", 16, rotation=90, "Computer Modern"))
display(p)

In [ ]:
LS_AC_over_r = autocorr_rel_err(RES["test1_AC"][:LS], RES["test1_AC"][:fom])
int_AC_over_r = autocorr_rel_err(RES["test1_AC"][:int], RES["test1_AC"][:fom])
ephec_AC_over_r = autocorr_rel_err(RES["test1_AC"][:ephec], RES["test1_AC"][:fom])
;

In [ ]:
plot(DATA["ro"], LS_AC_over_r, c=:black, marker=(:circle, 3.5, :black), label="opinf")
plot!(DATA["ro"], int_AC_over_r, c=:orange, marker=(:cross, 8, :orange), label="intrusive")
plot!(DATA["ro"], ephec_AC_over_r, c=:blue, markerstrokecolor=:blue, marker=(:rect, 3), ls=:dash, lw=2, label="ep-opinf")

plot!(majorgrid=true, legend=:topright)
xlabel!("reduced model dimension " * L" r")
ylabel!("error of normalized autocorrelation")
# yticks!([1e-0, 1e-1, 1e-2, 1e-3])
xticks!(DATA["ro"][1]:2:DATA["ro"][end])
plot!(left_margin=2mm)
plot!(fontfamily="Computer Modern", guidefontsize=13, tickfontsize=13,  legendfontsize=13)


## Test 2

In [ ]:
lags = 0:KSE_data.DS:Int(floor(KSE.Tdim));

In [ ]:
# FOM
foo = zeros(length(lags))
bar = length(TEST_DATA["ic_a_rand_out"])
prog = Progress(bar)
for i in 1:bar
    foo .+= analyze_autocorr(KSE, TEST_DATA["Xtest_out"], i, lags)[1]
    next!(prog)
end
TEST_RES["test2_AC"][:fom][1] = foo / bar


In [ ]:
# Least-squares
foo = zeros(length(lags), length(DATA["ro"]))
bar = length(TEST_DATA["ic_a_rand_out"])
prog = Progress(bar)
for i in 1:bar
    a = TEST_DATA["ic_a_rand_out"][i]
    b = TEST_DATA["ic_b_rand_out"][i]

    tmp = analyze_autocorr(OPS["op_LS"], KSE, DATA["Vr"], u0(a,b), DATA["ro"], KSE.integrate_FD, lags)
    for j in eachindex(DATA["ro"])
        foo[:,j] .+= tmp[j,1]
    end
    next!(prog)
end

for j in eachindex(DATA["ro"])
    TEST_RES["test2_AC"][:LS][j,1] = foo[:,j] / bar
end


In [ ]:
# intrusive
foo = zeros(length(lags), length(DATA["ro"]))
bar = length(TEST_DATA["ic_a_rand_out"])
prog = Progress(bar)
for i in 1:bar
    a = TEST_DATA["ic_a_rand_out"][i]
    b = TEST_DATA["ic_b_rand_out"][i]

    tmp = analyze_autocorr(OPS["op_int"], KSE, DATA["Vr"], u0(a,b), DATA["ro"], KSE.integrate_FD, lags)
    for j in eachindex(DATA["ro"])
        foo[:,j] .+= tmp[j,1]
    end
    next!(prog)
end

for j in eachindex(DATA["ro"])
    TEST_RES["test2_AC"][:int][j,1] = foo[:,j] / bar
end

In [ ]:
# ep-opinf
foo = zeros(length(lags), length(DATA["ro"]))
bar = length(TEST_DATA["ic_a_rand_out"])
prog = Progress(bar)
for i in 1:bar
    a = TEST_DATA["ic_a_rand_out"][i]
    b = TEST_DATA["ic_b_rand_out"][i]

    tmp = analyze_autocorr(OPS["op_ephec"], KSE, DATA["Vr"], u0(a,b), DATA["ro"], KSE.integrate_FD, lags)
    for j in eachindex(DATA["ro"])
        foo[:,j] .+= tmp[j,1]
    end
    next!(prog)
end

for j in eachindex(DATA["ro"])
    TEST_RES["test2_AC"][:ephec][j,1] = foo[:,j] / bar
end


In [ ]:
lags_t = collect(lags) .* KSE.Δt
idx = length(lags_t) ÷ 2
lout = @layout [grid(2,2)]
p = plot(layout=lout, size=(1000, 500))
for (plot_id, ri) in enumerate([1, 2, 5, 7])
    plot!(p[plot_id], lags_t[1:idx], TEST_RES["test2_AC"][:LS][ri][1:idx], c=:black, label="opinf")
    plot!(p[plot_id], lags_t[1:idx], TEST_RES["test2_AC"][:int][ri][1:idx], c=:orange, label="intrusive")
    plot!(p[plot_id], lags_t[1:idx], TEST_RES["test2_AC"][:ephec][ri][1:idx], c=:blue, ls=:dash, lw=2, label="ephec-opinf")
    # plot!(p[plot_id], lags_t, TEST_RES["test2_AC"][:epsic][ri], c=:purple, ls=:dot, label="epsic-opinf")
    # plot!(p[plot_id], lags_t, TEST_RES["test2_AC"][:epp][ri], c=:red, lw=1, ls=:dash, label="epp-opinf")
    plot!(p[plot_id], lags_t[1:idx], TEST_RES["test2_AC"][:fom][1][1:idx], c=:green, lw=1, ls=:dash, label="fom")
    plot!(p[plot_id], fontfamily="Computer Modern", guidefontsize=12, tickfontsize=13)
    plot!(p[plot_id], title="r = $(DATA["ro"][ri])", titlefontsize=14, titlefontfamily="Computer Modern")
    # xlabel!(p[plot_id], "lag")
    # ylabel!(p[plot_id], "normalized autocorrelation")
    yticks!(p[plot_id], [-0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0])

    if plot_id == 4
        plot!(p[plot_id], legend=:topright, legend_column=2, legendfontsize=12)
    else
        plot!(p[plot_id], legend=false)
    end
end
plot!(p[1], left_margin=9mm)
plot!(p[3], bottom_margin=9mm, left_margin=9mm)
plot!(p[4], bottom_margin=9mm)
annotate!(p[4], -20, -0.48, "time lag", annotationfontsize=16)
annotate!(p[1], -30, -0.4, Plots.text("normalized autocorrelation", 16, rotation=90, "Computer Modern"))
display(p)


In [ ]:
LS_AC_over_r = autocorr_rel_err(RES["test2_AC"][:LS], RES["test2_AC"][:fom])
int_AC_over_r = autocorr_rel_err(RES["test2_AC"][:int], RES["test2_AC"][:fom])
ephec_AC_over_r = autocorr_rel_err(RES["test2_AC"][:ephec], RES["test2_AC"][:fom])
;

In [ ]:
plot(DATA["ro"], LS_AC_over_r, c=:black, marker=(:circle, 3.5, :black), label="opinf")
plot!(DATA["ro"], int_AC_over_r, c=:orange, marker=(:cross, 8, :orange), label="intrusive")
plot!(DATA["ro"], ephec_AC_over_r, c=:blue, markerstrokecolor=:blue, marker=(:rect, 3), ls=:dash, lw=2, label="ep-opinf")

plot!(majorgrid=true, legend=:topright)
xlabel!("reduced model dimension " * L" r")
ylabel!("error of normalized autocorrelation")
# yticks!([1e-0, 1e-1, 1e-2, 1e-3])
xticks!(DATA["ro"][1]:2:DATA["ro"][end])
plot!(left_margin=2mm)
plot!(fontfamily="Computer Modern", guidefontsize=13, tickfontsize=13,  legendfontsize=13)